For the previous step - Big-FISH version: 0.5.0  

This notebook is used to compare all bigfish results to GT.  
However, because so many results were produced by a grid search, this was actually not ran as a notebook,  
but was ran on the cluster with a job for each original file.

#####  sh file that called this python script (mdc cluster, sun-grid):

```
#!/bin/sh
# Job script with qsub-options
##$ -pe smp 2
##$ -pe orte 4
#$ -V -N "BF_analysis"
#$ -l h_rt=80:00:00 -l h_vmem=10G -l h_stack=128M -l data -cwd

##$ -o bf-log.txt
##$ -e bf-log.txt

# export NSLOTS=8
# neccessary to prevent python error
export OPENBLAS_NUM_THREADS=4
# export NUM_THREADS=8
python bigfish_comparison.py --im_num $IM_NUM  &> log${IM_NUM}.txt
```  

##### pythos script that called the shell script:

```
import os

# Thr loop
for i in range(50):

	command = f'qsub -v IM_NUM={i} bigfish_analyse_results.sh'
	os.system(command)
```

In [ ]:
import sys
import os
import numpy as np
import csv
import copy
from scipy import spatial
from glob import glob
import pandas as pd
import argparse

In [ ]:
path = 'PATHTO/Selected_simulation'

In [ ]:
### im_num is a passed argument
## for each original image, all grid search results will be compared to GT here

parser = argparse.ArgumentParser(description='for cluster multiple jobs - im_num is given as arg')
parser.add_argument("--im_num")
args = parser.parse_args()
if args.im_num:
    im_num = int(args.im_num)
else:
    print('need to pass image number')

In [ ]:
## Leo's code:

def profile_detections(unmod, more_than): 

    min_dist = 2 

    distance_arr = [] 

    removedItems = True 
    euc_dist = 0 

    while (removedItems and len(more_than) != 0 and len(unmod) != 0 ): 
        #print("loop") 

        minDist = 10000 
        minIndexUnmod = -1 
        minIndexMore_Than = -1 
        counter = 0 
        kd_copy = copy.deepcopy(more_than) 
        kdtree = spatial.KDTree(kd_copy) 

        for item in unmod: 
            distance,index = kdtree.query(item) # a new KD tree is made 
            if ( distance < minDist ): 
                minDist = distance 
                minIndexUnmod = counter 
                minIndexMore_Than = index 
                #print(minDist, counter, item) 
            counter = counter + 1 

        if ( minDist < min_dist): # if less than min dist 
            more_than = np.delete(more_than, minIndexMore_Than, axis = 0 ) # delete mod ind 
            unmod = np.delete(unmod,minIndexUnmod, axis = 0) #delete unmod ind 
            #print(len(more_than),distance) # sanity checkd 
            removedItems = True 
            distance_arr.append(minDist) # if we want to extrat stat ig 

        else: 
            removedItems = False 
    if (len(distance_arr) >0): 
        euc_dist = np.mean(np.asarray(distance_arr)) 

    return(len(unmod), len(more_than), euc_dist) 

In [ ]:

all_paths = glob(os.path.join(path, '*','FishQuant_result*.csv')) 

all_paths = [p for p in all_paths if ("embryo" not in p) and ("3000spo" not in p) and
 (('Empty Bg Density Range Sigxy 2 SigZ 2' not in p) or ('Poiss_300spots_bg_200_2_I_300_0_img2.tif' not in p))] 

loc_files = glob(os.path.join(path, '*','*.loc'))
loc_files = [l for l in loc_files if "3000spo" not in l] 
loc_file = loc_files[im_num]

all_paths = [f for f in all_paths if loc_file.split('/')[-2] in f and loc_file.split('/')[-1][:-4] in f]

df = pd.read_csv(loc_file, sep = "\s+", header=None) 
gt_spots = df.to_numpy()[:,:-1] - np.array([0.5,0.5,1])

loc_diffs = []
loc_all_files = []

for i,p in enumerate(all_paths): 

    num_lines = sum(1 for line in open(p))

    if ("_30spots" in p and num_lines<60) or ("_300spots" in p and num_lines<500):
    
        detected_spots = pd.read_csv(os.path.join(path,p))[["y","x","z"]].to_numpy()
    
        diff_results = list(profile_detections(gt_spots, detected_spots))
        loc_diffs.append([str(d) for d in diff_results])
        loc_all_files.append(p)


In [ ]:
with open(f'bigfish_diff_results_{im_num}.txt', 'w') as f:
    for i,d in enumerate(loc_diffs):
        f.write(loc_all_files[i])    
        f.write(', ')
        f.write(",".join(d))
        f.write('\n')